<a href="https://colab.research.google.com/github/elemi10/7506-TP-Org-de-datos/blob/7506-Trabajo-Practico-1/Equipo_Kung_Fu_Pandas_TP1_7506_Org_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP1 - Real or Not?**

###  **Anexo: Análisis de Modelo de Clasificación**

![Banner_TP1.png](https://drive.google.com/uc?id=1BPA2RF1SDm9bTs1xZfVUa1VQn932E3wr)


In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
downloaded = drive.CreateFile({'id':'1RAGDjlzJ6spO5Sq8_x3UTIvxLhKAUBEt'}) # replace the id with id of file you want to access
downloaded.GetContentFile('train.csv') 

### Importamos **Librerias**

In [0]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

### Levantramos el archivo **train.csv**

In [81]:
df_train = pd.read_csv(r'train.csv', usecols=['keyword','location','text','target']) 
df_train.head()

,keyword,location,text,target
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Preprocesamiento de datos

In [82]:
# Verifico que no haya instancias nulas o filas completas nulas
df_train=df_train.dropna(how="all")
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7552 non-null   object
 1   location  5080 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [83]:
# Normalizo el dataframe los registros NaN con texto
df_train=df_train.fillna({'keyword': 'sin keyword',\
                   'location': 'sin location'})
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7613 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7613 non-null   object
 1   location  7613 non-null   object
 2   text      7613 non-null   object
 3   target    7613 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 297.4+ KB


In [84]:
# Eliminamos los tweets duplicados considerando que su cantidad no tiene impacto en el análisis.
df_train=df_train.drop_duplicates('text')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7503 entries, 0 to 7612
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   keyword   7503 non-null   object
 1   location  7503 non-null   object
 2   text      7503 non-null   object
 3   target    7503 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 293.1+ KB


In [85]:
# Verificamos la existencia de valores nulos
df_train.isnull().any()

keyword     False
location    False
text        False
target      False
dtype: bool

### **Análisis & tokenización**

In [0]:
# Función de tokenización
import string
punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

In [0]:
# Creo la serie 'text_vector'
df_train['text_vector']=df_train['text'].apply(tokenize)
df_train.head(2)

In [0]:
# Verifico integridad de DF
df_train.info()

In [0]:
# Descargo e importo lista de stopwords para filtrar
import nltk
nltk.download('stopwords')

In [0]:
from nltk.corpus import stopwords
stopwords.fileids()

In [0]:
# Asigno en la variable 'stop' la categoria de stopword por la que voy a realizar el filtro
stop=stopwords.words('english')

In [0]:
def filtro(text_vector):
  text_vector_filtrado = []
  for palabra in text_vector:
    if palabra not in stop:
      text_vector_filtrado.append(palabra)
  return(text_vector_filtrado)

In [0]:
# Tokens filtrados por stopwords en inglés
df_train['text_vector_filtrado']=df_train['text_vector'].apply(filtro)
df_train['text_vector_filtrado'].head()

In [0]:
# Asigno en una nueva columna el largo del vector 'text_vector'
df_train['elem_vector']=df_train['text_vector'].str.len()
df_train.head(2)

In [0]:
# Cuento los elementos del vector filtrado 'text_vector_filtrado'
df_train['elem_vector_filtrado']=df_train['text_vector_filtrado'].str.len()
df_train.head(2)

In [0]:
# Verifico estructura del DF
df_train.info()

### Preproceso y categorización de la variable ***Keyword***

In [0]:
# "Limpiamos" la serie de caracteres especiales
df_train['keyword']=df_train['keyword'].str.replace('%20','_')
# Verificamos actualización
df_train['keyword'].unique()

### Generación de **diccionario**

In [0]:
# Listo para generar un diccionario y traducir la serie 'keyword'
keyword_list=df_train['keyword'].unique().tolist()
print(keyword_list)

In [0]:
#1 Genero lista en espaniol
keyword_list_esp=['sin_keyword','en llamas','accidente','replica','accidente_avion','ambulancia','aniquilado','aniquilacion','apocalipse','armageddon','ejercito','incendio_intencional','piromano','ataque','atacado','avalancha','combate/batalla/lucha','bioterror','bioterrorismo','resplandecer/arder','resplandeciente/ardiente','hemorragia','exploto','plaga','ventisca','sangre','sangriento','estallido','bolsa_de_cadaver','embolsado_de_cadaver','bolsas_de_cadaver','bomba','bombardeado','bombardeo','colapso_de_puente','incendio_de_edificios','edificios_en_llamas','quemado','quemando/incendiando','incendio_edificios','incendio_de_matorrales/arbustos','bajas/muertes/perdidas','baja/muerte/perdida','catastrofe','catastrofico','emergencia_quimica','caida_acantilado','colapso','colapsado','colisionar','colisiono','colision','choque','estrellado','aplastamiento','aplastado','toque_de_queda','ciclon','da¤o','peligro','muerto','muerte','muertes','escombros','diluvio','inundado','demoler','demolido','demolicion','descarrilar','descarrilado','descarrilamiento','desolado','desolacion','destruir','destruido','destruccion','detonado','detonacion','devastado','devastacion','desastre','dislacado/desplazado','sequia','ahogar','ahogado','ahogamiento/ahogandose','tormenta_polvo','terremoto','electrocutar','electrocutado','emergencia','plan_de_emergencia','servicio_de_emergencia','engullido','epicentro','evacuar','evacuado','evacuacion','explotar','exploto/detonado/estallado','explosion','testigo_ocular','hambruna','fatal','fatalidades','fatalidades','miedo','fuego','camion_bombero','primeros_en_responder','llamas','aplastado','inundacion','inundaciones','inundaciones','incendio_forestal','incendio_forestales','granizo','tormenta_granizo','da¤o','peligro','peligroso','ola_de_calor','infierno','secuestro','secuestrador','secuestro','rehen','rehenes','huracan','herido/lesionado','heridas/lesiones','herida/lesion','inundado','inundacion','deslisamiento_tierra','lava','rayo','golpe_fuerte/estampido_fuerte','asesinato_en_serie','asesino_en_serie','masacre','caos','fusion','militar','alud_de_lodo','desastre_natural','desastre_nuclear','reactor_nuclear','obliterar/destruir/eliminar','borrado/destruido/eliminado','obliteracion/destruccion/eliminacion','derrame_de_petroleo','brote','pandemonio/confusion/caos','panico','tener_panico','polic¡a','cuarentena','en_cuarentena','emergencia_radiacion','tormenta','arrasado','refugiados','rescate','rescatado','rescatadores','disturbio','disturbios','escombros','ruina','tormenta_de_arena','grito','gritando','gritos','sismico','sumidero/pozo','hundiendose','sirena','sirenas','humo','tormenta_de_nieve','tormenta','camilla','fallo_estructural','bomba_suicida','suicida','bombardeo_suicida','hundido','sobrevivir','sobrevivio','sobreviviente','terrorismo','terrorista','amenaza','trueno','tormenta_electrica','tornado','tragedia','atrapado','trauma','traumatizado','problema','tsunami','tornado/torbellino','tifon','transtorno','tormenta_violenta','volcan','zona_de_guerra','arma','armas','torbellino','incendios_forestales','incendio_forestal','tormenta_de_viento','moretoneado','moretones','naufragio','restos','naufrago']
print(keyword_list_esp)

In [0]:
#1 Genero el diccionario eng-esp de la serie 'Keyword'
keyword_dicc={}
y=0
for x in keyword_list:
  keyword_dicc[x]=keyword_list_esp[y]
  y=y+1
  #print(keyword_dicc)

print(keyword_dicc)

In [0]:
# Mapeo los valores de las key del diccionario keyword_dicc
df_train['keyword_esp']=df_train['keyword'].map(keyword_dicc)
df_train.head(2)

In [0]:
# Verifico el top 20 de las categorias mas frecuentes para la serie keyword 
df_train['keyword_esp'].value_counts().sort_values(ascending=False).head(20)

In [0]:
# Hacemos un groupby de las categorias keyword con tweets verdaderos y falsos ('target')
df_agrupado=df_train.groupby(['keyword_esp','tweet_type']).size().sort_values(ascending=False)
df_agrupado

In [0]:
# Otro groupby de la categorias contando ocurrencias de tweets y de caracteres que los conforman
df_agrupado_ocurrencias=df_train.groupby(['keyword_esp']).agg({'target':['count'],'caracteres_tweet':['max','min','mean']})
df_agrupado_ocurrencias

### → **Scikit Learn**: Ejecución de **One Hot Encoding** para clasificar

In [0]:
# Dummifico las categorías
df_train_dummies=pd.get_dummies(df_train.keyword)
df_train_dummies.head(2)

In [0]:
# Hago un join con df_train y df_train_dummies
df_train_merged = pd.concat([df_train,df_train_dummies],axis='columns')
df_train_merged.head(2)

In [0]:
# Elimino las columnas 'keyword_esp' y 'sin keyword'
df_train_merged_final=df_train_merged.drop(['keyword_esp','sin keyword'],axis='columns')
df_train_merged_final.head(2)

In [0]:
# Exporto a .csv para verificar estructura
df_train_merged_final.to_csv('columna_texto_vector_filtrado.csv')

In [0]:
print(df_train_merged_final['text_vector_filtrado'])

Vectorizo con el feature **CountVectorizer**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [0]:
# Defino los documentos/vector
documents=(df_train_merged_final['text'])
documents

In [0]:
vectorizer=CountVectorizer(stop_words='english')
vectors=vectorizer.fit_transform(documents)
names=vectorizer.get_feature_names()
#names

In [0]:
doc_array=vectorizer.transform(documents).toarray()
doc_array

### Implementación de **BoW** O **Bag of Word**

In [0]:
frecuency_matrix=pd.DataFrame(data=doc_array, columns=names)
frecuency_matrix

### Set de datos de **Pruebas** y de **Entrenamiento**

In [0]:
# Levanto libreria para dividir los conjuntos de entrenamiento y test
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test= train_test_split(df_train_merged_final['text'],\
                                                   df_train_merged_final['target'], random_state=1)

In [0]:
print('Cant total de filas en el set de datos: {}'.format(df_train_merged_final.shape[0]))

In [0]:
print('Cant de filas en el set de datos de entrenamiento: {}'.format(X_train.shape[0]))

In [0]:
print('Cant de filas en el set de datos de pruebas: {}'.format(X_test.shape[0]))

### Ejecución de BoW para procesar los datos de pruebas

In [0]:
count_vector=CountVectorizer()

In [0]:
training_data=count_vector.fit_transform(X_train)
training_data

In [0]:
testing_data=count_vector.transform(X_test)
testing_data

### Implementación de Naives Bayes Multinomial para clasificar

In [0]:
naives_bayes=MultinomialNB()
naives_bayes.fit(training_data, y_train)
naives_bayes.fit(testing_data, y_test)

In [0]:
# Realizamos alguna prediccion con el conjunto de pruebas
predicciones=naives_bayes.predict(testing_data)

### Evaluación del modelo

In [0]:
# Para todas las mediciones - cuyo rango de probabilidad es entre o y 1 - tener una puntuación cercana a 1
# es buen estimador del comportamiento del modelo
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score

In [0]:
print("Métrica de Exactitud del Modelo: ",format(accuracy_score(y_test,predicciones)))

In [0]:
print("Métrica de Precisión del Modelo: ",format(precision_score(y_test,predicciones)))

In [0]:
print("Sensibilidad o recall del Modelo: ",format(f1_score(y_test, predicciones)))